### Step 1: Import packages

In [109]:
import pandas as pd
import numpy as np
import re

### Step 2: Define working directories

In [110]:
path_raw_data = 'C:/users/lbros/documents/mids/w207/final_project/raw_data/'
path_clean_data = 'C:/users/lbros/documents/mids/w207/final_project/clean_data/'

### Step 3: Read data

#### Read raw ratings data

In [111]:
# load ratings dataframe
ratings_df = pd.read_csv(path_raw_data + 'ratings.csv')

In [112]:
# print dataframe shape
ratings_df.shape

(26024289, 4)

In [113]:
# print dataframe columns
ratings_df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [114]:
# inspect first five rows
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


#### Read raw links data

In [115]:
# load links dataframe
links_df = pd.read_csv(path_raw_data + 'links.csv')

In [116]:
# print dataframe shape
links_df.shape

(45843, 3)

In [117]:
# print dataframe columns
links_df.columns

Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')

In [118]:
# inspect first five rows
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


#### Read clean movies data

In [119]:
# load movies dataframe
movies_df = pd.read_csv(path_clean_data + 'movies_temp.csv')

In [120]:
# print dataframe shape
movies_df.shape

(45418, 179)

In [121]:
# print dataframe columns
movies_df.columns

Index(['imdb_id', 'id', 'adult', 'belongs_to_collection', 'budget',
       'originally_english', 'overview', 'popularity', 'production_companies',
       'production_countries',
       ...
       'zu', 'canceled', 'in-production', 'planned', 'post-production',
       'released', 'rumored', 'cast_names', 'crew_names', 'description'],
      dtype='object', length=179)

In [122]:
# inspect first five rows
movies_df.head()

,imdb_id,id,adult,belongs_to_collection,budget,originally_english,overview,popularity,production_companies,production_countries,...,zu,canceled,in-production,planned,post-production,released,rumored,cast_names,crew_names,description
0,tt3155794,315946,0,0,0,0,photo sequenc rare transit venu face sun one f...,0.480371,unknown,fr,...,0,0,0,0,0,1,0,unknown,pjcjanssen,astronomi silent film venu planet 19th centuri...
1,tt2221420,194079,0,0,0,1,salli gardner gallop one earliest silent film ...,0.327841,palo alto stock farm,us,...,0,0,0,0,0,1,0,salliegardner gilbertdomm,eadweardmuybridge lelandstanford,silent film hors chronophotographi stop motion...
2,tt5459794,426903,0,0,0,1,individu photograph run buffalo shot rapid suc...,0.229221,unknown,us,...,0,0,0,0,0,1,0,unknown,eadweardmuybridge,run buffalo photographi short black white
3,tt2075247,159897,0,0,0,0,last remain product le princ lpcc type16 16len...,1.184891,unknown,us,...,0,0,0,0,0,1,0,unknown,louisaiméaugustinleprince,short black white walk
4,tt1758563,96882,0,0,0,0,last remain film le princ lpccp type1 mkii sin...,0.212768,whitley partners,gb,...,0,0,0,0,0,1,0,adolpheleprince,louisaiméaugustinleprince,musician silent film


### Step 4: Pre-process ratings data

#### Add imdbId field to ratings_df

In [123]:
# set movieId as index on links_df
links_df.set_index('movieId', inplace=True)

In [124]:
# set userId and movieId as indexes on ratings_df
ratings_df.set_index(['userId', 'movieId'], inplace=True)

In [125]:
# insert imdbId and tmdbId fields on ratings_df left joining with links_df on movieId index
ratings_df = ratings_df.join(links_df, on='movieId', how='left').reset_index()

In [126]:
# drop tmdbId
ratings_df.drop('tmdbId', axis=1, inplace=True)

In [127]:
# print dataframe shape
ratings_df.shape

(26024289, 5)

In [128]:
# print dataframe columns
ratings_df.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'imdbId'], dtype='object')

In [129]:
# inspect first five rows
ratings_df.head()

,userId,movieId,rating,timestamp,imdbId
0,1,110,1.0,1425941529,112573
1,1,147,4.5,1425942435,112461
2,1,858,5.0,1425941523,68646
3,1,1221,5.0,1425941546,71562
4,1,1246,5.0,1425941556,97165


#### Binarize ratings

Define **binarize_ratings** function

In [130]:
def binarize_ratings(ratings_df, threshold=4):
    
    '''Binarize ratings:
    - 1 for ratings equal or above threshold
    - 0 for ratings below threshold
    params: ratings_df, threshold
    return: ratings_df with binarized ratings
    '''
    
    # binarize ratings field
    ratings_df['rating'] = ratings_df['rating'].apply(lambda x: 1 if x >= 4 else 0)
    
    return ratings_df

The ratings frequency are **equally split between <4 and >=4**.

In [131]:
# apply binarize function to ratings_df
ratings_df = binarize_ratings(ratings_df)

In [132]:
# inspect binarized ratings frequency
ratings_df.rating.value_counts(normalize=True)

0    0.501168
1    0.498832
Name: rating, dtype: float64

### Step 5: Evaluate keys to join ratings with movies data

#### Evaluate movieId

The movieId **do not seem to be an effective key** to join the datasets.

In [133]:
rated_movies = ratings_df.movieId.unique()
print('There are {} unique movies rated in the ratings_df.'.format(len(rated_movies)))

There are 45115 unique movies rated in the ratings_df.


In [134]:
repr_movies = movies_df.id.unique()
print('There are {} unique movies represented in the movies_df.'.format(len(repr_movies)))

There are 45413 unique movies represented in the movies_df.


In [135]:
rated_not_in_repr = rated_movies[~np.isin(rated_movies, repr_movies)]
print('There are {} movies in the ratings_df not in the movies_df.'.format(len(rated_not_in_repr)))
rated_not_in_repr

There are 37552 movies in the ratings_df not in the movies_df.


array([  1221,   2918,   4878, ..., 165649, 171051, 171221], dtype=int64)

In [136]:
repr_not_in_rated = repr_movies[~np.isin(repr_movies, rated_movies)]
print('There are {} movies in the movies_df not in the ratings_df.'.format(len(repr_not_in_rated)))
repr_not_in_rated

There are 37850 movies in the movies_df not in the ratings_df.


array([315946, 194079, 426903, ...,  38700, 299782,  76600], dtype=int64)

#### Try with imdb_id instead

##### Inspect and clean imdb_id field in movies_df

In [137]:
# no missing values on movies_df imdbd_id...
print('Missing values: ', movies_df['imdb_id'].isna().sum())
# ...and no unknown values
unknown = movies_df['imdb_id'][~movies_df['imdb_id'].str.contains('^tt')]
print('Unknow values: ', len(unknown))

Missing values:  0
Unknow values:  0


In [138]:
def clean_imdb_id(movies_df):
    """Drop rows with unknow values from df,
    remove 'tt' and cast into numerical values"""
    
    # remove the 'tt' in each line
    p = re.compile(r'[tt]+')
    movies_df['imdb_id'] = [p.sub('', x) for x in movies_df['imdb_id'].tolist()]
    
    # convert to numeric format to allow join with ratings_df
    movies_df['imdb_id'] = pd.to_numeric(movies_df['imdb_id'])
    
    return movies_df

In [139]:
# apply this function to convert the imdb_id field to numbers
movies_df = clean_imdb_id(movies_df)

##### Evaluate imdb_id

In [140]:
rated_movies = ratings_df.imdbId.unique()
print('There are {} unique movies rated in the ratings_df.'.format(len(rated_movies)), '\n')
print('The minimum imdb_id is {}.'.format(rated_movies.min()))
print('The maximum imbd_id is {}.'.format(rated_movies.max()))

There are 45115 unique movies rated in the ratings_df. 

The minimum imdb_id is 1.
The maximum imbd_id is 7158814.


In [141]:
repr_movies = movies_df.imdb_id.unique()
print('There are {} unique movies represented in the movies_df.'.format(len(repr_movies)), '\n')
print('The minimum imdb_id is {}.'.format(repr_movies.min()))
print('The maximum imbd_id is {}.'.format(repr_movies.max()))

There are 45413 unique movies represented in the movies_df. 

The minimum imdb_id is 1.
The maximum imbd_id is 7158814.


In [142]:
rated_not_in_repr = rated_movies[~np.isin(rated_movies, repr_movies)]
print('There are {} movies in the ratings_df not in the movies_df.'.format(len(rated_not_in_repr)), '\n')

There are 487 movies in the ratings_df not in the movies_df. 



In [143]:
repr_not_in_rated = repr_movies[~np.isin(repr_movies, rated_movies)]
print('There are {} movies in the movies_df not in the ratings_df.'.format(len(repr_not_in_rated)))

There are 785 movies in the movies_df not in the ratings_df.


#### Drop non-matching movies from both datasets

In [144]:
def drop_non_matching_movies(ratings_df, movies_df):
    """Drop rows with non matching movies"""
    
    # unique movies in ratings_df
    rated_movies = ratings_df.imdbId.unique()
    # unique movies in movies_df
    repr_movies = movies_df.imdb_id.unique()
    # find the intersection
    commom_movies = np.intersect1d(rated_movies, repr_movies)
    # filter ratings_df with movies in the intersection
    ratings_df = ratings_df[np.isin(ratings_df.imdbId.values, commom_movies)]
    # filter movies_df with movies in the intersection
    movies_df = movies_df[np.isin(movies_df.imdb_id.values, commom_movies)]
    
    return ratings_df, movies_df

In [145]:
# apply this function to drop non-matching movies from datasets
ratings_df, movies_df = drop_non_matching_movies(ratings_df, movies_df)

In [146]:
# print remaining imdb_id duplicates (if any)
print('Number of duplicated imdb_id: ', movies_df[movies_df['imdb_id'].duplicated()].shape[0])

Number of duplicated imdb_id:  5


In [147]:
# Drop remaining imdb_id duplicates, keeping the last indexed entry
movies_df = movies_df[~movies_df['imdb_id'].duplicated(keep='last')]

We dropped **54,665 rows** (out of 26,024,289) in the **ratings_df**.

In [148]:
print('There are now {} records in the ratings_df.'.format(ratings_df.shape[0]))

There are now 25969624 records in the ratings_df.


In [149]:
print('There are now {} unique movies rated in the ratings_df.'.format(len(ratings_df.imdbId.unique())))

There are now 44628 unique movies rated in the ratings_df.


We dropped **790 rows** (out of 45,418) in the **movies_df**.

In [150]:
print('There are now {} records in the movies_df.'.format(movies_df.shape[0]))

There are now 44628 records in the movies_df.


In [151]:
print('There are now {} unique movies rated in the movies_df.'.format(len(movies_df.imdb_id.unique())))

There are now 44628 unique movies rated in the movies_df.


#### Replace vote_count in the movies_df by the counting of votes in the ratings_df

In [152]:
ratings_by_movie = ratings_df.groupby('imdbId').count().rating
movies_df = movies_df.set_index('imdb_id').join(ratings_by_movie).reset_index()
movies_df['vote_count'] = movies_df['rating']
movies_df.drop('rating', axis=1, inplace=True)

#### Replace vote_average in the movies_df by the averaging of ratings in the ratings_df

In [ ]:
mean_rating_by_movie = ratings_df.groupby('imdbId').mean().rating
movies_df = movies_df.set_index('imdb_id').join(mean_rating_by_movie).reset_index()
movies_df['vote_average'] = movies_df['rating']
movies_df.drop('rating', axis=1, inplace=True)

### Step 6: Filter users with a minimum number of ratings

Define **filter_users** function

In [153]:
def filter_users(ratings_df, min_ratings=30, min_pct=0.2):
    
    '''Filter users with num_ratings equal or above [min_ratings]
    and at least [min_pct] of movies rated as 0 or 1
    params: ratings_df, min_rating
    return: filtered_ratings_df
    '''
    
    # filter userIDs with count of ratings equal or above min_ratings
    temp_ratings_df = ratings_df.groupby('userId', sort=False).filter(lambda x: len(x) >= min_ratings)
    
    # filter userIDs with less than min_pct of 0 or 1 ratings
    aux_df = temp_ratings_df.groupby(['userId', 'rating'], sort=False).count()['movieId'] / temp_ratings_df.groupby(['userId'], sort=False).count()['movieId']
    aux_df = aux_df.reset_index('rating')
    aux_df = aux_df[aux_df['rating']==0]
    aux_df['min_pct'] = (aux_df['movieId'] > min_pct) & (aux_df['movieId'] < (1-min_pct))
    aux_df = aux_df.drop(['rating', 'movieId'], axis=1)
    temp_ratings_df = temp_ratings_df.set_index('userId')
    temp_ratings_df = temp_ratings_df.join(aux_df, how='left')
    temp_ratings_df = temp_ratings_df[temp_ratings_df['min_pct']==True]
    temp_ratings_df = temp_ratings_df.drop(['min_pct'], axis=1)
    filtered_ratings_df = temp_ratings_df.reset_index()
    
    return filtered_ratings_df

We filtered users with **less than 30 ratings** and **less than 20% of ratings 0 or 1** resulting in a filtered dataset with **115,715 users** (out of 270,881), **41,664 movies** (out of 44,628) and **20,568,389 ratings** (out of 25,969,624).

In [154]:
filtered_ratings_df = filter_users(ratings_df, min_ratings=30)
print('Filtered number of users:',filtered_ratings_df.userId.unique().shape[0])
print('Total number of users:', ratings_df.userId.unique().shape[0])

Filtered number of users: 115715
Total number of users: 270881


In [155]:
print('Filtered number of movies:',filtered_ratings_df.imdbId.unique().shape[0])
print('Total number of movies:',ratings_df.imdbId.unique().shape[0])

Filtered number of movies: 41664
Total number of movies: 44628


In [156]:
print('Filtered number of ratings:',filtered_ratings_df.shape[0])
print('Total number of ratings:',ratings_df.shape[0])

Filtered number of ratings: 20568389
Total number of ratings: 25969624


### Step 7: Split data

#### Split users into dev and test sets

In [157]:
def user_split(ratings_df, dev_size=5000, random_state=100):
    
    '''Split users into development and test sets'''
    
    # define random seed
    np.random.seed(seed=random_state)
    # randomly pick [dev_size] users
    unique_users = ratings_df['userId'].unique()
    dev = np.random.choice(unique_users, size=dev_size, replace=False)
    # split users into dev and test based on picked users
    dev_users = ratings_df[np.isin(ratings_df['userId'], dev)]
    test_users = ratings_df[~np.isin(ratings_df['userId'], dev)]
    
    return dev_users, test_users

In [158]:
# apply user_split to filtered_ratings_df
dev_users, test_users = user_split(filtered_ratings_df)

In [159]:
print('Development set has {} unique users and {} ratings in total.'.format(len(dev_users['userId'].unique()), dev_users.shape[0]))

Development set has 5000 unique users and 911733 ratings in total.


In [160]:
print('Test set has {} unique users and {} ratings in total.'.format(len(test_users['userId'].unique()), test_users.shape[0]))

Test set has 110715 unique users and 19656656 ratings in total.


#### Hold last rating by user for evaluation

In [161]:
def hold_last_out(user_data):
    
    '''Split ratings by user into train and test sets using the hold
    last out method. Test contains the last rated movie by user while
    training contains all the other rated movies.
    '''
    
    # find the indexes correspondent to maximum timestamp by user
    idx_test = user_data.groupby('userId', sort=False).idxmax()['timestamp']
    # filter test data with idx_test
    test = user_data.loc[idx_test]
    # drop idx_test to get train data
    train = user_data.drop(idx_test, axis=0)
    
    return train, test

In [162]:
# apply hold_last_out to dev_users
dev_train, dev_test = hold_last_out(dev_users)

In [163]:
print('Dev_train set has {} unique users and {} ratings in total.'.format(len(dev_train['userId'].unique()), dev_train.shape[0]))

Dev_train set has 5000 unique users and 906733 ratings in total.


In [164]:
print('Dev_test set has {} unique users and {} ratings in total.'.format(len(dev_test['userId'].unique()), dev_test.shape[0]))

Dev_test set has 5000 unique users and 5000 ratings in total.


In [165]:
# apply hold_last_out to test_users
test_train, test_test = hold_last_out(test_users)

In [166]:
print('Test_train set has {} unique users and {} ratings in total.'.format(len(test_train['userId'].unique()), test_train.shape[0]))

Test_train set has 110715 unique users and 19545941 ratings in total.


In [167]:
print('Test_test set has {} unique users and {} ratings in total.'.format(len(test_test['userId'].unique()), test_test.shape[0]))

Test_test set has 110715 unique users and 110715 ratings in total.


#### Final adjustments

In [168]:
# filter usable columns
usable_columns = ['userId', 'imdbId', 'rating']
dev_train = dev_train[usable_columns]
dev_test = dev_test[usable_columns]
test_train = test_train[usable_columns]
test_test = test_test[usable_columns]

In [169]:
# rename imdbId
dev_train.columns = ['userId', 'imdb_id', 'rating']
dev_test.columns = ['userId', 'imdb_id', 'rating']
test_train.columns = ['userId', 'imdb_id', 'rating']
test_test.columns = ['userId', 'imdb_id', 'rating']

### Step 8: Save clean and filtered datasets to a csv file

In [170]:
# write cleaned, filtered and splitted ratings dataset to csv files
dev_train.to_csv('C:/Users/lbros/Documents/MIDS/W207/final_project/clean_data/dev_train.csv')
dev_test.to_csv('C:/Users/lbros/Documents/MIDS/W207/final_project/clean_data/dev_test.csv')
test_train.to_csv('C:/Users/lbros/Documents/MIDS/W207/final_project/clean_data/test_train.csv')
test_test.to_csv('C:/Users/lbros/Documents/MIDS/W207/final_project/clean_data/test_test.csv')

In [171]:
# write filtered movie dataset to a csv file
movies_df.to_csv('C:/Users/lbros/Documents/MIDS/W207/final_project/clean_data/movies_final.csv')